## Merging data
Merging two datasets into one CSV file.
1. TOUR_CAP_NAT
2. isoc_ci_dev_i

### 1. Download and read raw data

In [27]:
import requests
from io import BytesIO
import pandas as pd

def get_raw_data(name: str):
    url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/{name}"
    response = requests.get(url, params={"format":"tsv","compressed":"true"})
    df = pd.read_csv(
        BytesIO(response.content), 
        sep="\t", 
        compression="gzip"
    )
    return df
    
# read '.tsv.gz' format files with pandas
tour_cap_nat_df = get_raw_data("tour_cap_nat")
isoc_ci_dev_i_df = get_raw_data("isoc_ci_dev_i")

# check first 5 rows
tour_cap_nat_df.head()

,"freq,accomunit,unit,nace_r2,geo\TIME_PERIOD",1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,"A,BEDPL,NR,I551,AT",650559,654127,653586,651003,650020,646125,640199,633601,584889,...,600342,601945,609393,615541,607010,608836,601641,610655,610755,:
1,"A,BEDPL,NR,I551,BE",93710,86472,101804,108205,108811,114887,117299,117513,116297,...,129307,129232,129456,136406,137484,139565,139252,144015,148173,:
2,"A,BEDPL,NR,I551,BG",262966,154350,124680,127450,114400,118112,107111,99953,112002,...,279090,281869,293494,286219,288027,233549,255058,294183,299081,:
3,"A,BEDPL,NR,I551,CH",269759,267067,266150,265136,264824,264983,262471,261482,260290,...,273507,271710,275203,274792,273849,:,280471,:,287412,:
4,"A,BEDPL,NR,I551,CY",57602,61871,68066,72211,74671,77259,83517,83288,85161,...,84426,83251,84977,86252,89200,89912,89626,87227,83871,:


In [28]:
isoc_ci_dev_i_df.head()

,"freq,ind_type,indic_is,unit,geo\TIME_PERIOD",2016,2018,2021,2023
0,"A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AL",:,: u,:,:
1,"A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT",37.41,42.45,39.62,40.34
2,"A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BA",:,47.02,:,:
3,"A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BE",41.45,35.53,28.30,28.36
4,"A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BG",40.80 u,39.81 u,: u,30.98 u


### 2. Processing SDMX-TSV file
Eurostat offers TSV foramt file in SDMX 2.1 and 3.0 APIs.

It is required to process SDMX-TSV file in proper manners.

[reference: EuroStat](https://wikis.ec.europa.eu/display/EUROSTATHELP/API+-+FAQ+-+TSV+data+format)

In [29]:
def process_special_values(df_: pd.DataFrame):
    # Copy original
    df = df_.copy()
    
    # Replace '', ':', '..' as None
    #     (SDMX often uses ':' or '..' as empty value)
    df.replace(['', ':', '..'], None, inplace=True)
    
    # OBS_FLAG == "u" or "bu" -> considered as missing data
    obs_flag_replace_target = ["u", "bu"]
    df["OBS_FLAG"] = df["OBS_FLAG"].replace(obs_flag_replace_target, None)
    
    # Country codes(= geo) "EA", "EU27_2007", "EU27_2020", "EU28" are same as missing data
    geo_replace_target = ["EA", "EU27_2007", "EU27_2020", "EU28"]
    df["geo"] = df["geo"].replace(geo_replace_target, None)
    
    return df


def process_sdmx(df: pd.DataFrame):
    """Process SDMX format data"""
    # Split the first column name into multiple names
    col_names = df.columns[0].split(',')
    col_names[-1], time_period_col = col_names[-1].split('\\')
    
    # Split the first column into multiple columns
    first_col_df = df.iloc[:, 0].str.split(',', expand=True)
    
    # Set column names
    first_col_df.columns = col_names
    # Merge first column data with original df
    df_merged = pd.concat([first_col_df, df.iloc[:, 1:]], axis=1)
    
    # Melt the data to get tidy format
    df_melted = df_merged.melt(id_vars=col_names, var_name=time_period_col, value_name="observ")
    # Remove useless space in time_period column
    df_melted[time_period_col] = df_melted[time_period_col].str.strip()
    
    # Split observations into OBS_VALUE & OBS_FLAG
    observ = df_melted["observ"].str.split(' ', expand=True)
    observ.columns = ["OBS_VALUE", "OBS_FLAG"]
    df_final = pd.concat([df_melted.drop(columns="observ"), observ], axis=1)
    
    # Strip all object columns
    for col in df_final.select_dtypes('object').columns:
        df_final[col] = df_final[col].str.strip()
    
    # Process special values
    df_final = process_special_values(df_final)
    
    return df_final


tour_cap_nat_df_tidy = process_sdmx(tour_cap_nat_df)
print("Tidied TOUR_CAP_NAT shape:", tour_cap_nat_df_tidy.shape)
tour_cap_nat_df_tidy.head()

Tidied TOUR_CAP_NAT shape: (33390, 8)


,freq,accomunit,unit,nace_r2,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,A,BEDPL,NR,I551,AT,1990,650559,None
1,A,BEDPL,NR,I551,BE,1990,93710,None
2,A,BEDPL,NR,I551,BG,1990,262966,None
3,A,BEDPL,NR,I551,CH,1990,269759,None
4,A,BEDPL,NR,I551,CY,1990,57602,None


In [30]:
isoc_ci_dev_i_df_tidy = process_sdmx(isoc_ci_dev_i_df)
print("Tidied isoc_ci_dev_i shape:", isoc_ci_dev_i_df_tidy.shape)
isoc_ci_dev_i_df_tidy.head()

Tidied isoc_ci_dev_i shape: (324268, 8)


,freq,ind_type,indic_is,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AL,2016,None,None
1,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT,2016,37.41,None
2,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BA,2016,None,None
3,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BE,2016,41.45,None
4,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,BG,2016,40.80,None


### 3. Fetch "Rows to use" requirements

#### 3-1. TOUR_CAP_NAT
Rows to use: You should only use rows where:
- "accomunit" is "BEDPL"
- "unit" is "NR"
- "nace_r2" is "I551"
- "TIME_PERIOD" is "2016"

In [31]:
tour_cap_nat_df_filtered = tour_cap_nat_df_tidy.loc[
    (tour_cap_nat_df_tidy["accomunit"] == "BEDPL")
    & (tour_cap_nat_df_tidy["unit"] == "NR")
    & (tour_cap_nat_df_tidy["nace_r2"] == "I551")
    & (tour_cap_nat_df_tidy["TIME_PERIOD"] == "2016")
].reset_index(drop=True)

print("Filtered TOUR_CAP_NAT shape:", tour_cap_nat_df_filtered.shape)
tour_cap_nat_df_filtered.head()

Filtered TOUR_CAP_NAT shape: (44, 8)


,freq,accomunit,unit,nace_r2,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,A,BEDPL,NR,I551,AT,2016,601945,None
1,A,BEDPL,NR,I551,BE,2016,129232,None
2,A,BEDPL,NR,I551,BG,2016,281869,None
3,A,BEDPL,NR,I551,CH,2016,271710,None
4,A,BEDPL,NR,I551,CY,2016,83251,None


#### 3-2. isoc_ci_dev_i
Rows to use: 
- "ind_type" is "IND_TOTAL"
- "indic_is" is "I_IUG_TV"
- "unit" is "PC_IND"
- "TIME_PERIOD" is "2016"


In [32]:
isoc_ci_dev_i_df_filtered = isoc_ci_dev_i_df_tidy.loc[
    (isoc_ci_dev_i_df_tidy["ind_type"] == "IND_TOTAL")
    & (isoc_ci_dev_i_df_tidy["indic_is"] == "I_IUG_TV")
    & (isoc_ci_dev_i_df_tidy["unit"] == "PC_IND")
    & (isoc_ci_dev_i_df_tidy["TIME_PERIOD"] == "2016")
].reset_index(drop=True)

print("Filtered isoc_ci_dev_i shape:", isoc_ci_dev_i_df_filtered.shape)
isoc_ci_dev_i_df_filtered.head()

Filtered isoc_ci_dev_i shape: (35, 8)


,freq,ind_type,indic_is,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,A,IND_TOTAL,I_IUG_TV,PC_IND,AT,2016,14.78,None
1,A,IND_TOTAL,I_IUG_TV,PC_IND,BE,2016,9.89,None
2,A,IND_TOTAL,I_IUG_TV,PC_IND,BG,2016,3.71,None
3,A,IND_TOTAL,I_IUG_TV,PC_IND,CY,2016,6.41,None
4,A,IND_TOTAL,I_IUG_TV,PC_IND,CZ,2016,5.32,None


### 4. Merge datasets and preprocess
Combine two public data sets into a single CSV file that contains 3 columns: “Country Code”, “Percentage of individuals” and “Number of Bed-places”.

The “Country Code” column should be distinct and should not contain missing data.

In [47]:
# Country Code(=geo) should not contain missing data
tour_cap_nat_df_filtered.dropna(subset="geo", ignore_index=True, inplace=True)
isoc_ci_dev_i_df_filtered.dropna(subset="geo", ignore_index=True, inplace=True)

merged_df = pd.merge(
    tour_cap_nat_df_filtered, 
    isoc_ci_dev_i_df_filtered,
    on="geo", 
    how="outer",
    suffixes=["1", "2"]
)[["geo", "OBS_VALUE1", "OBS_VALUE2"]]

merged_df.columns = [
    "Country Code", 
    "Number of Bed-places",
    "Percentage of individuals",
]

print("Is 'Country Code' unique? :", merged_df["Country Code"].is_unique)
merged_df.head()

Is 'Country Code' unique? : True


,Country Code,Number of Bed-places,Percentage of individuals
0,AT,601945,14.78
1,BE,129232,9.89
2,BG,281869,3.71
3,CH,271710,NaN
4,CY,83251,6.41


In [57]:
from pathlib import Path

path_to_save = Path.cwd().parent/"data"/"combined.csv"
merged_df.to_csv(path_to_save, index=False)